<center><img src="car.jpg" width=500></center>


Insurance companies invest a lot of time and money into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries insurance it is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

(`Source: https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf`) 

Knowing all of this, On the Road car insurance have requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to identify the single feature that results in the best performing model, as measured by accuracy, so they can start with a simple model in production.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table detailing the column names and descriptions below.



## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-25</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

In [19]:
# Import required modules
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit

# Start coding!

In [20]:
df=pd.read_csv('car_insurance.csv')
df.head()

,id,age,gender,driving_experience,education,income,credit_score,vehicle_ownership,vehicle_year,married,children,postal_code,annual_mileage,vehicle_type,speeding_violations,duis,past_accidents,outcome
0,569520,3,0,0-9y,high school,upper class,0.629027,1.0,after 2015,0.0,1.0,10238,12000.0,sedan,0,0,0,0.0
1,750365,0,1,0-9y,none,poverty,0.357757,0.0,before 2015,0.0,0.0,10238,16000.0,sedan,0,0,0,1.0
2,199901,0,0,0-9y,high school,working class,0.493146,1.0,before 2015,0.0,0.0,10238,11000.0,sedan,0,0,0,0.0
3,478866,0,1,0-9y,university,working class,0.206013,1.0,before 2015,0.0,1.0,32765,11000.0,sedan,0,0,0,0.0
4,731664,1,1,10-19y,none,working class,0.388366,1.0,before 2015,0.0,0.0,32765,12000.0,sedan,2,0,1,1.0


In [21]:
# Check for missing values in the dataframe
missing_values = df.isnull().sum()

# Drop rows with any missing values
df_clean = df.dropna()

# Display the number of rows dropped and the shape of the cleaned dataframe
missing_values, df.shape, df_clean.shape

(id                       0
 age                      0
 gender                   0
 driving_experience       0
 education                0
 income                   0
 credit_score           982
 vehicle_ownership        0
 vehicle_year             0
 married                  0
 children                 0
 postal_code              0
 annual_mileage         957
 vehicle_type             0
 speeding_violations      0
 duis                     0
 past_accidents           0
 outcome                  0
 dtype: int64,
 (10000, 18),
 (8149, 18))

In [22]:
# Define the formula for logistic regression
formula = 'outcome ~ age + gender + C(driving_experience) + C(education) + C(income) + credit_score + vehicle_ownership + C(vehicle_year) + married + children + annual_mileage + C(vehicle_type) + speeding_violations + duis + past_accidents'

# Fit the logistic regression model using the cleaned dataframe
model = logit(formula, data=df_clean).fit()

# Store the results
results = model.summary()

Optimization terminated successfully.
         Current function value: 0.364437
         Iterations 8


In [23]:
# List of features to model individually (excluding the target 'outcome' and 'id', 'postal_code')
features = [
    'age', 'gender', 'driving_experience', 'education', 'income', 'credit_score',
    'vehicle_ownership', 'vehicle_year', 'married', 'children', 'annual_mileage',
    'vehicle_type', 'speeding_violations', 'duis', 'past_accidents'
]

# List to store model results
single_feature_results = []

# Fit one logistic regression model per feature
for feature in features:
    # Use C() for categorical/object features
    if df_clean[feature].dtype == 'object':
        formula = f'outcome ~ C({feature})'
    else:
        formula = f'outcome ~ {feature}'
    model = logit(formula, data=df_clean).fit(disp=0)
    single_feature_results.append(model.summary())

# single_feature_results now contains the summary of each model
single_feature_results

[<class 'statsmodels.iolib.summary.Summary'>
 """
                            Logit Regression Results                           
 Dep. Variable:                outcome   No. Observations:                 8149
 Model:                          Logit   Df Residuals:                     8147
 Method:                           MLE   Df Model:                            1
 Date:                Tue, 02 Sep 2025   Pseudo R-squ.:                  0.1733
 Time:                        22:41:07   Log-Likelihood:                -4177.4
 converged:                       True   LL-Null:                       -5052.9
 Covariance Type:            nonrobust   LLR p-value:                     0.000
                  coef    std err          z      P>|z|      [0.025      0.975]
 ------------------------------------------------------------------------------
 Intercept      0.6551      0.044     15.048      0.000       0.570       0.740
 age           -1.1193      0.031    -36.635      0.000      -1.179   

In [24]:
from sklearn.metrics import confusion_matrix

# List to store accuracy of each model
single_feature_accuracies = []

for i, feature in enumerate(features):
    # Use C() for categorical/object features
    if df_clean[feature].dtype == 'object':
        formula = f'outcome ~ C({feature})'
    else:
        formula = f'outcome ~ {feature}'
    model = logit(formula, data=df_clean).fit(disp=0)
    # Predict probabilities
    pred_probs = model.predict(df_clean)
    # Convert probabilities to binary predictions (threshold 0.5)
    preds = (pred_probs >= 0.5).astype(int)
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(df_clean['outcome'], preds).ravel()
    # Calculate accuracy as (TN + TP) / (TN + TP + FN + FP)
    acc = (tn + tp) / (tn + tp + fn + fp)
    single_feature_accuracies.append(acc)

# Display as a DataFrame for clarity
import pandas as pd
pd.DataFrame({'feature': features, 'accuracy': single_feature_accuracies})

,feature,accuracy
0,age,0.774083
1,gender,0.688796
2,driving_experience,0.778132
3,education,0.688796
4,income,0.739968
5,credit_score,0.706590
6,vehicle_ownership,0.739232
7,vehicle_year,0.688796
8,married,0.688796
9,children,0.688796


In [25]:
# Exclude "id" from features if present
filtered_features = [f for f in features if f != "id"]
filtered_accuracies = [acc for f, acc in zip(features, single_feature_accuracies) if f != "id"]

# Find the index of the highest accuracy among filtered features
best_idx = pd.Series(filtered_accuracies).idxmax()
best_feature = filtered_features[best_idx]
best_accuracy = filtered_accuracies[best_idx]

# Store as a DataFrame
best_feature_df = pd.DataFrame({
    "best_feature": [best_feature],
    "best_accuracy": [best_accuracy]
})

best_feature_df

,best_feature,best_accuracy
0,driving_experience,0.778132
